In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

## 한경 컨센서스 리포트 크롤링


In [2]:
fromDate = '2022-05-21'
toDate = '2022-06-21'
page = 2
reportType='ALL'
gradeCode='ALL'
changePrices='ALL'
searchType='ALL'
url = f'https://markets.hankyung.com/api/v1/consensus/search/report?page={page}&reportType={reportType}&fromDate={fromDate}&toDate={toDate}&gradeCode={gradeCode}&changePrices={changePrices}&searchType={searchType}'

In [3]:
response = requests.get(url)
response

<Response [200]>

In [4]:
last_page = response.json()['last_page']
datas = response.json()['data']

In [5]:
type(datas)

dict

In [6]:
datas = []
for page in range(1, 3) :
    url = f'https://markets.hankyung.com/api/v1/consensus/search/report?page={page}&reportType={reportType}&fromDate={fromDate}&toDate={toDate}&gradeCode={gradeCode}&changePrices={changePrices}&searchType={searchType}'
    response = requests.get(url)
    data = response.json()['data']
    if type(data) == dict :
        data = data.values()
#     print(pd.DataFrame(data))
    datas.append(pd.DataFrame(data))

In [7]:
df = pd.concat(datas)
df.reset_index(inplace=True)
df.dropna(axis=1, inplace=True)

In [8]:
# 필요한 데이터 : REPORT_IDX, PUBLISH_CODE, OFFICE_NAME, INDUSTRY_CODE, INDUSTRY_NAME, REPORT_TITLE, REPORT_WRITER, REPORT_FILEPATH, REPORT_DATE
result_df = df[['REPORT_IDX', 'PUBLISH_CODE', 'OFFICE_NAME', 'INDUSTRY_CODE', 'INDUSTRY_NAME', 'REPORT_TITLE', 'REPORT_WRITER', 'REPORT_FILEPATH', 'REPORT_DATE']]
result_df.head()

,REPORT_IDX,PUBLISH_CODE,OFFICE_NAME,INDUSTRY_CODE,INDUSTRY_NAME,REPORT_TITLE,REPORT_WRITER,REPORT_FILEPATH,REPORT_DATE
0,610539,0015,메리츠증권,072,일반전기전자,전기전자 - 디스플레이 6월 하반월 LCD 패널가,김선우,https://markets.hankyung.com/pdf/2022/06/9f57b...,2022-06-21
1,610552,0012,유진투자증권,,,Eugene’s FICC Update (2022.6.20),"허재환, 김지나",https://markets.hankyung.com/pdf/2022/06/cfa33...,2022-06-21
2,610551,0012,유진투자증권,,,자산진단 키트 - 진격의 Fed,방인성,https://markets.hankyung.com/pdf/2022/06/d0138...,2022-06-21
3,610550,0029,유안타증권,,,선물옵션시장 지표,정인지,https://markets.hankyung.com/pdf/2022/06/ccf98...,2022-06-21
4,610549,4015,우리은행,,,"Daily Forex Live 리스크 Off --> On, 1,290원 하향이탈 시도","민경원, 신승연",https://markets.hankyung.com/pdf/2022/06/9735e...,2022-06-21


### 함수화

In [9]:
# 크롤링을 위한 함수화
def hankyung_data(fromDate, toDate, reportType='ALL', gradeCode='ALL',changePrices='ALL',searchType='ALL') :
    url = f'https://markets.hankyung.com/api/v1/consensus/search/report?page=1&reportType={reportType}&fromDate={fromDate}&toDate={toDate}&gradeCode={gradeCode}&changePrices={changePrices}&searchType={searchType}'
    response = requests.get(url)
    last_page = response.json()['last_page']
    datas = []
    for page in range(1, last_page+1) :
        url = f'https://markets.hankyung.com/api/v1/consensus/search/report?page={page}&reportType={reportType}&fromDate={fromDate}&toDate={toDate}&gradeCode={gradeCode}&changePrices={changePrices}&searchType={searchType}'
        response = requests.get(url)
        data = response.json()['data']
        if type(data) == dict :
            data = data.values()
        datas.append(pd.DataFrame(data))
    df = pd.concat(datas)
    df.reset_index(inplace=True)
    df.dropna(axis=1, inplace=True)
    result_df = df[['REPORT_IDX', 'PUBLISH_CODE', 'OFFICE_NAME', 'INDUSTRY_CODE', 'INDUSTRY_NAME', 'REPORT_TITLE', 'REPORT_WRITER', 'REPORT_FILEPATH', 'REPORT_DATE']]
    
    return result_df

In [10]:
df = hankyung_data('2022-05-21', '2022-06-21')
len(df)

1958

## 데이터 다운로드 후 OCR 데이터 확인

In [ ]:
# 인터넷 pdf 파일 저장
import requests
URL = "https://markets.hankyung.com/pdf/2022/06/9f57b436d1977954380a7ee69c552170"
  
file = requests.get(URL, stream = True)
  
with open("Python.pdf","wb") as pdf:
    for chunk in file.iter_content(chunk_size=1024):
        if chunk:
            pdf.write(chunk)